In [2]:
import numpy as np
import cv2

In [3]:
# read depth image
depth_scale = 0.00012498664727900177
depth_img = cv2.imread('depth.png')
dpt = depth_img[:, :, 2] + depth_img[:, :, 1] * 256
dpt = dpt * depth_scale

# read seg image
seg = cv2.imread('seg.png')[...,0]  # 255: fore ground, 0: background

# read intrinsics and extrinsics
K = np.load('intrinsic.npy')
print(K)

[[415.69219382   0.         320.        ]
 [  0.         415.69219382 240.        ]
 [  0.           0.           1.        ]]


In [24]:
# task1: convert depth image to point cloud
def depth2pc(depth, seg, K):
    # ------------TODO---------------
    # compute point cloud from depth image
    # for-loop is not allowed!!
    valid_corr = np.where(seg == 255)
    valid_depth = depth[valid_corr]
    points_2d = np.array([valid_corr[1], valid_corr[0], np.ones(len(valid_corr[0]))])
    pc = ((np.linalg.inv(K) @ points_2d) * valid_depth).T
    # ------------TODO --------------
    return pc

partial_pc = depth2pc(dpt, seg, K)

# For debug and submission
np.savetxt('../results/pc_from_depth.txt', partial_pc)

In [33]:
# task2: compute one-way chamfer distance to the complete shape
full_pc = np.loadtxt('aligned_full_pc.txt')

def random_sample(pc, num):
    permu = np.random.permutation(pc.shape[0])
    return pc[permu][:num]

partial_pc_sampled = random_sample(partial_pc, 2048)
full_pc_sampled = random_sample(full_pc, 2048)

# -----------TODO---------------
# implement one way chamfer distance
distance = np.sum((partial_pc_sampled[:, np.newaxis, :] - full_pc_sampled) ** 2, axis = 2) ** 0.5
one_way_CD = np.sum(distance.min(axis = 1)) / distance.shape[0]
# -----------TODO---------------
print('one way chamfer distance: ', one_way_CD)

# For submission
np.savetxt('../results/one_way_CD.txt', [[one_way_CD]])

one way chamfer distance:  0.009997255112500852
